# Producer Kafka

In [15]:
import time
from kafka import KafkaProducer

kafka_url = 'kafka:9092'

# Démarrer le producteur Kafka
producer = KafkaProducer(bootstrap_servers=kafka_url, value_serializer=lambda v: str(v).encode('utf-8'))

# Envoi de messages

In [16]:
topic = 'topic-example'

for i in range(10):
    message = f'Message {i}'
    producer.send(topic, value=message)
    print(f'Envoyé: {message}')
    time.sleep(1)  # Pause pour simuler un flux

producer.flush()
producer.close()

Envoyé: Message 0
Envoyé: Message 1
Envoyé: Message 2
Envoyé: Message 3
Envoyé: Message 4
Envoyé: Message 5
Envoyé: Message 6
Envoyé: Message 7
Envoyé: Message 8
Envoyé: Message 9


# Consumer Kafka

In [7]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    topic,
    bootstrap_servers=[kafka_url],
    auto_offset_reset='earliest',
    consumer_timeout_ms=1000,
    value_deserializer=lambda value: value.decode('utf-8'),
)

print("Liste des messages reçus:")
for message in consumer:
    print(f'Reçu: {message.value}')

consumer.close()

Liste des messages reçus:
Reçu: Message 0
Reçu: Message 1
Reçu: Message 2
Reçu: Message 3
Reçu: Message 4
Reçu: Message 5
Reçu: Message 6
Reçu: Message 7
Reçu: Message 8
Reçu: Message 9


# Création de topic en ligne de commande :
1) me connecter au conteneur kafka :
```bash
# trouver le nom du conteneur kafka
docker ps
```
sortie similaire à :
```
CONTAINER ID   IMAGE                           COMMAND                  CREATED             STATUS             PORTS                                                                     NAMES
2071b4e63acd   pythonproject2-pytest           "/opt/bitnami/script…"   About an hour ago   Up About an hour   0.0.0.0:8888->8888/tcp, :::8888->8888/tcp                                 pythonproject2-pytest-1
5c683d94c827   bitnamilegacy/kafka:3.6.1       "/opt/bitnami/script…"   2 hours ago         Up About an hour   0.0.0.0:9092->9092/tcp, :::9092->9092/tcp                                 pythonproject2-kafka-1
a69f975bd00e   bitnamilegacy/zookeeper:3.9.1   "/opt/bitnami/script…"   2 hours ago         Up About an hour   2888/tcp, 3888/tcp, 0.0.0.0:2181->2181/tcp, :::2181->2181/tcp, 8080/tcp   pythonproject2-zookeeper-1
```
Ici le nom du conteneur kafka est `pythonproject2-kafka-1`
```bash
# se connecter au conteneur kafka en ligne de commande
docker exec -it <container_name> bash
# donc avec le nom
docker exec -it pythonproject2-kafka-1 bash
```

Avec docker compose, on peut aussi faire :
```bash
# ou kafka est le nom du service déclarer dans le docker-compose.yml
docker compose exec kafka bash
```

Créer les topics :
```bash
kafka-topics.sh --create --topic topic-example --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1
```

# Exercice : lire un fichier et envoyer chaque ligne dans un topic Kafka

In [9]:
kafka_url = 'kafka:9092'

# Démarrer le producteur Kafka
producer = KafkaProducer(bootstrap_servers=kafka_url, value_serializer=lambda v: str(v).encode('utf-8'))
topic2 = 'topic-example-file'
with open('data/test.csv', 'r') as file:
    lines = file.readlines()
    for line in lines:
        producer.send(topic2, value=line.strip())
        print(f'Envoyé: {line.strip()}')
        time.sleep(0.1)  # Pause pour simuler un flux

    producer.flush()

producer.close()

Envoyé: test;asdf
Envoyé: test;qwer
Envoyé: test;zxcv


In [10]:
from kafka import KafkaConsumer

consumer = KafkaConsumer(
    topic2,
    bootstrap_servers=[kafka_url],
    auto_offset_reset='earliest',
    consumer_timeout_ms=1000,
    value_deserializer=lambda value: value.decode('utf-8'),
)

print("Liste des messages reçus:")
for message in consumer:
    print(f'Reçu: {message.value}')

consumer.close()

Liste des messages reçus:
Reçu: test;asdf
Reçu: test;qwer
Reçu: test;zxcv


# Administration de Kafka depuis python

In [12]:
from kafka.admin import KafkaAdminClient, NewTopic

admin_client = KafkaAdminClient(
    bootstrap_servers=kafka_url,
    client_id='test_admin'
)

topics = admin_client.list_topics()
print("Topics existants:", topics)

Topics existants: ['topic-example', 'topic-example-file']


In [13]:
new_topic_name = 'topic-from-python'
if new_topic_name not in topics:
    topic = NewTopic(name=new_topic_name, num_partitions=1, replication_factor=1)
    admin_client.create_topics(new_topics=[topic], validate_only=False)
    print(f'Topic "{new_topic_name}" créé.')
else:
    print(f'Topic "{new_topic_name}" existe déjà.')

# Supprimer le topic
admin_client.delete_topics([new_topic_name])

Topic "topic-from-python" créé.


DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='topic-from-python', error_code=0)])

# Supprimer toutes les données d'un topic :
1) Supprimer et recréer le topic (plus simple)
2) Modifier la configuration du topic pour réduire le temps de rétention des messages

In [14]:
from kafka.admin import ConfigResource, ConfigResourceType

resource = ConfigResource(
    ConfigResourceType.TOPIC,
    topic2,
    configs={ "retention.ms": "0" }  # 0 milli-secondes
)
admin_client.alter_configs([resource])
print(f'Configuration de rétention du topic "{topic2}" modifiée pour supprimer les messages immédiatement.')

# Remettre la configuration par défaut (7 jours)
resource = ConfigResource(
    ConfigResourceType.TOPIC,
    topic2,
    configs={ "retention.ms": str(7 * 24 * 60 * 60 * 1000) }  # 7 jours
)
admin_client.alter_configs([resource])
print(f'Configuration de rétention du topic "{topic2}" remise à 7 jours.')

Configuration de rétention du topic "topic-example-file" modifiée pour supprimer les messages immédiatement.
Configuration de rétention du topic "topic-example-file" remise à 7 jours.
